<a href="https://colab.research.google.com/github/cellatlas/human/blob/master/markers/uterus/markers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet anndata statsmodels
!pip install -q gget
!pip install -q git+https://github.com/sbooeshaghi/ec


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.2/25.2 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import pandas as pd
import numpy as np
from ec.utils import write_markers

In [3]:
!wget --quiet "https://figshare.com/ndownloader/files/28846980"

In [4]:
!unzip 28846980

Archive:  28846980
  inflating: TS_Uterus.h5ad          
  inflating: __MACOSX/._TS_Uterus.h5ad  


In [5]:
# Get valid gene names
!wget -O genes.txt https://caltech.box.com/shared/static/4r5prhstoq2j1lk8l56bpwqqvfj54q2u.txt
genes_list = pd.read_csv('genes.txt', header = None)[0].values

--2023-03-18 06:03:46--  https://caltech.box.com/shared/static/4r5prhstoq2j1lk8l56bpwqqvfj54q2u.txt
Resolving caltech.box.com (caltech.box.com)... 74.112.186.144
Connecting to caltech.box.com (caltech.box.com)|74.112.186.144|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/4r5prhstoq2j1lk8l56bpwqqvfj54q2u.txt [following]
--2023-03-18 06:03:46--  https://caltech.box.com/public/static/4r5prhstoq2j1lk8l56bpwqqvfj54q2u.txt
Reusing existing connection to caltech.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://caltech.app.box.com/public/static/4r5prhstoq2j1lk8l56bpwqqvfj54q2u.txt [following]
--2023-03-18 06:03:46--  https://caltech.app.box.com/public/static/4r5prhstoq2j1lk8l56bpwqqvfj54q2u.txt
Resolving caltech.app.box.com (caltech.app.box.com)... 74.112.186.144
Connecting to caltech.app.box.com (caltech.app.box.com)|74.112.186.144|:443... connected.
HTTP request sent, awaiting response... 3

# Uterus

In [6]:
organ = "uterus"
paper = "https://doi.org/10.1126/science.abl4896"
table = "https://figshare.com/ndownloader/files/28846980" # anndata

header = [
    
    {
        'table_link' : table
    }
]

In [7]:
import anndata
import numpy as np

In [8]:
#@title dexpress.py

from scipy.stats import ks_2samp, ttest_ind
import numpy as np
import pandas as pd
import logging
import sys

import numpy as np
from statsmodels.stats.multitest import multipletests

def split_by_target(mat, targets, target, axis=0):
    """
    Split the rows of mat by the proper assignment
    
    mat = ndarray
    targets, length is equal to number of components (axis=0) or features (axis=1)
    target is a singular element from unique(assignments/features) 
    """
    if axis==0 and len(targets) != mat.shape[axis]: return -1
    if axis==1 and len(targets) != mat.shape[axis]: return -1
        
    mask = targets == target
    
    if axis==0:
        t_mat = mat[mask]  # target matrix
        c_mat = mat[~mask] # complement matrix
    elif axis==1:
        t_mat = mat[:, mask]  # target matrix
        c_mat = mat[:, ~mask] # complement matrix
    
    return (t_mat, c_mat)

def correct_pval(pval, nfeat, method):
    if method == 'bonferroni':
        pval_corr = np.minimum(pval*nfeat[:,None], 1.0)
    return pval_corr

def nd(arr):
    return np.asarray(arr).reshape(-1)
def correct_pvals(pvals):
    r = np.empty(np.shape(pvals))
    r[:] = np.nan
    for pidx, p in enumerate(pvals):
        mask = ~np.isnan(p)


    return r

def dexpress(mat, components, features, assignments, **kwargs):
    """
    mat: ndarray
    components: nparray
    features: nparray
    assignments: nparray, length is equal to length of components
    """
    nan_cutoff = kwargs.get("nan_cutoff", 0)
    
    unique = np.unique(assignments)
        
    ncomp = components.shape[0]
    nfeat = features.shape[0]
    nuniq = unique.shape[0]
    
    pval = np.empty((nuniq, nfeat)) # unique assignments by features 
    pval_corr = np.empty((nuniq, nfeat)) # unique assignments by features 
    stat = np.empty((nuniq, nfeat)) # unique assignments by features 
    es = np.empty((nuniq, nfeat)) # unique assignments by features 
    nfeatures = np.empty((nuniq))
    
    pval.fill(np.nan)
    pval_corr.fill(np.nan)
    stat.fill(np.nan)
    es.fill(np.nan)
    nfeatures.fill(np.nan)
    

    for tidx, t in enumerate(unique):
        logging.info("{} of {} assignments: {}".format(tidx+1, nuniq, t))
        t_mat, c_mat = split_by_target(mat, assignments, t, axis=0)
        
        pvalue = np.nan
        statistic = np.nan
        effect_size = np.nan

        # Keep only the genes for which it is in at least nan_cutoff*ncellsinclusters
        mask = np.asarray((t_mat>0).sum(axis=0) > nan_cutoff*t_mat.shape[0]).reshape(-1)

        t_mat = t_mat[:, mask]
        c_mat = c_mat[:, mask]
        # added

        # added
        if mask.sum()>0:
            test = ttest_ind(t_mat, c_mat, nan_policy="propagate", equal_var=False)
            pvalue = test.pvalue
            statistic = test.statistic
            effect_size = nd(t_mat.mean(axis=0)) - nd(c_mat.mean(axis=0))
            nfeatures[tidx] = mask.sum()
                
        pval[tidx,:][mask] = pvalue
        stat[tidx,:][mask] = statistic
        es[tidx,:][mask] = effect_size
    return (pval, stat, es, nfeatures)

def make_table(assignments, features, p_raw, p_corr, es):
    unique = np.unique(assignments)
    markers = pd.DataFrame(index=unique, columns=["name", "p_raw", "p_corr", "es"])

    for uidx, u in enumerate(unique):
        markers.loc[u]["name"] = features
        markers.loc[u]["p_raw"] = p_raw[uidx]
        markers.loc[u]["p_corr"] = p_corr[uidx]
        markers.loc[u]["es"] = es[uidx]

    markers = markers.apply(pd.Series.explode).reset_index()
    markers = markers[markers.notnull()["p_raw"]]
    return markers



In [9]:
gene = anndata.read_h5ad("TS_Uterus.h5ad")
gene.layers["log1p"] = np.log1p(gene.layers["raw_counts"])

In [10]:
mat           = gene.layers["log1p"].todense()
components    = gene.obs.index.values
features      = gene.var.gene_symbol.values
assignments   = gene.obs.free_annotation.values

unique = np.unique(assignments)
nan_cutoff = 0.9 # of elements in cluster
corr_method = "bonferroni"


p_raw, stat, es, nfeat = dexpress(mat, components, features, assignments, nan_cutoff=nan_cutoff)
p_raw = p_raw/2
p_corr = correct_pval(p_raw, nfeat, corr_method)

s = stat

markers_gene = make_table(assignments, features, p_raw, p_corr, es)

# convert the 0 pvalues to the smallest possible float
markers_gene["p_corr"][markers_gene.eval("p_corr == 0").values] = sys.float_info.min
markers_gene = markers_gene.query("es > 0")

In [11]:
df = markers_gene.rename(columns={"index": "celltype", "name": "gene"})

In [12]:
bidx = df['gene'].isin(genes_list)
print(f'Filtered {np.sum(~bidx)} out of {len(bidx)} genes')
df = df[bidx]


Filtered 12 out of 4162 genes


In [13]:
df.head()

,celltype,gene,p_raw,p_corr,es
755,B lymphocytes,RPL11,0.005209,1.0,0.362733
1280,B lymphocytes,SMAP2,0.000092,0.018762,1.342987
1423,B lymphocytes,RPS8,0.030904,1.0,0.238902
1483,B lymphocytes,UQCRH,0.165226,1.0,0.228354
3073,B lymphocytes,MCL1,0.483782,1.0,0.010871


In [14]:
min_mean = 10
max_pval = 0.05
min_lfc = 0.75
max_gene_shares = 5

# filter by criteria
dfc = df.query(f"p_corr <= {max_pval} & es >= {min_lfc}")

# mask out genes that are shared between max_gene_shares cell type
non_repeat_genes = dfc["gene"].value_counts()[dfc["gene"].value_counts() < max_gene_shares].index.values

m = dfc[dfc.gene.isin(non_repeat_genes)].sort_values('es', ascending = True)

# max number to sample is equal to the min number of genes across all celltype
n_sample = m["celltype"].value_counts().min()

# get n_sample genes
markers = m.groupby('celltype').tail(n_sample)
markers_dict = markers.groupby("celltype")["gene"].apply(lambda x: list(x)).to_dict()

In [15]:
m.celltype.value_counts()

Cycling epithelial cells           1085
Lymphatic endothelium               370
Endometrial stromal fibbroblast     130
Cycling immune cells                130
Unciliated epithelium                63
Macrophages                          63
Ciliated epithelium                  57
Pericyte                             42
Endothelium                          39
Vascular smooth muscle               38
Myometrial smooth muscle             23
Uterine NK cells                     12
B lymphocytes                         6
T lymphocytes                         5
Name: celltype, dtype: int64

In [16]:
markers.celltype.value_counts()

T lymphocytes                      5
B lymphocytes                      5
Endothelium                        5
Pericyte                           5
Uterine NK cells                   5
Cycling immune cells               5
Myometrial smooth muscle           5
Lymphatic endothelium              5
Vascular smooth muscle             5
Ciliated epithelium                5
Macrophages                        5
Endometrial stromal fibbroblast    5
Unciliated epithelium              5
Cycling epithelial cells           5
Name: celltype, dtype: int64

In [17]:
write_markers("markers.txt", markers_dict, header)

In [18]:
!cat markers.txt

# 				https://figshare.com/ndownloader/files/28846980
B lymphocytes	EZR,SMAP2,HLA-DPA1,CD74,CD79A
Ciliated epithelium	EPCAM,ELF3,WFDC2,CLDN3,CAPS
Cycling epithelial cells	CLDN3,CLDN4,ELF3,WFDC2,MMP7
Cycling immune cells	CCL4,PCLAF,CORO1A,HMGB2,HIST1H4C
Endometrial stromal fibbroblast	COL1A2,COL3A1,MMP11,SFRP4,COL1A1
Endothelium	CD93,NDRG1,CSF3,SERPINE1,TM4SF1
Lymphatic endothelium	ENG,KLF2,RAMP2,A2M,IGFBP7
Macrophages	RNASE1,CCL3,FCER1G,TYROBP,IL1B
Myometrial smooth muscle	MYL9,TPM2,ACTA2,MYH11,TAGLN
Pericyte	ACTA2,C11orf96,RGS5,ADIRF,TAGLN
T lymphocytes	SARAF,ARHGDIB,CD3D,TSC22D3,CXCR4
Unciliated epithelium	KRT8,EPCAM,UCA1,ELF3,CLDN4
Uterine NK cells	TSC22D3,CD7,AREG,NKG7,GNLY
Vascular smooth muscle	MYL9,C11orf96,MYH11,TAGLN,ADIRF


In [19]:
# Gzip and split anndata object to store locally
# ! gzip TS_uterus.h5ad
# !split -C 99m --numeric-suffixes TS_Uterus.h5ad.gz TS_Uterus.h5ad.gz